# Analyse de données RNA-Seq issues de patients atteints de mélanome uvéal

### Introduction
Dans ce hackathon, nous tenterons de reproduire les résultats de [1].
Les données originales proviennent de [2], où les auteurs ont séquencé l’ARN d’échantillons de patients atteints de mélanome uvéal. Ces patients appartiennent à deux groupes:

1. patients mutés pour le gène SF3B1 (facteur d’épissage) et
2. patients non mutés pour ce gène. 

Dans [2], les auteurs n’ont pas trouvé de différences d’épissage entre les patients mutés et les non mutés (bien qu’il s’agisse d’un facteur d’épissage).
En revanche, les auteurs de [1] ont réanalysé les mêmes données, et ont trouvé les gènes suivants épissés différemment entre les deux conditions: 
* ABCC5, 
* CRNDE,
* UQCC,
* GUSBP11,
* ANKHD1,
* ADAM12.

### Données

Les données sont accessibles sur SRA
* Patients mutés 
  * [SRR628582](https://www.ncbi.nlm.nih.gov/sra/?term=SRR628582)
  * [SRR628583](https://www.ncbi.nlm.nih.gov/sra/?term=SRR628583)
  * [SRR628584](https://www.ncbi.nlm.nih.gov/sra/?term=SRR628584)
* Patients non mutés
  * [SRR628585](https://www.ncbi.nlm.nih.gov/sra/?term=SRR628585)
  * [SRR628586](https://www.ncbi.nlm.nih.gov/sra/?term=SRR628586)
  * [SRR628587](https://www.ncbi.nlm.nih.gov/sra/?term=SRR628587)
  * [SRR628588](https://www.ncbi.nlm.nih.gov/sra/?term=SRR628588)
  * [SRR628589](https://www.ncbi.nlm.nih.gov/sra/?term=SRR628589)

### Objectif

Nous tenterons de retrouver les gènes différentiellement épissés entre les deux conditions, en utilisant un système de workflow et les outils suivants:
* [STAR](https://github.com/alexdobin/STAR),
* [DEXSeq](http://bioconductor.org/packages/release/bioc/html/DEXSeq.html),
* [SRA toolkit](https://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?view=software)
* [R](https://cran.r-project.org/)
* [Samtools](http://www.htslib.org/download/)

Le workflow peut se présenter comme cela:

![RNA Workflow](hackathon_1_workflow.svg)

## Références
* [1] [Furney et al. (2013)](https://www.ncbi.nlm.nih.gov/pubmed/23861464)
* [2] [Harbour et al. (2013)](https://www.ncbi.nlm.nih.gov/pubmed/23313955)

### Configuration de l'environnement IPython

In [14]:
import os
from urllib.request import urlretrieve

In [15]:
EXE ={
    "fastq-dump": "sratoolkit.2.8.2-1-ubuntu64/bin/fastq-dump",
    "STAR": "bin/STAR"
}

In [20]:
from ipyparallel import Client, require
rc = Client()
par = rc.load_balanced_view()
par["EXE"] = EXE

TypeError: 'LoadBalancedView' object does not support item assignment

# Genome

Le téléchargement et le traitement du génome est traité dans [Genome.ipynb](Genome.ipynb)

In [12]:
%run "Genome.ipynb"

chr1 ready
chr2 ready
chr3 ready
chr4 ready
chr5 ready
chr6 ready
chr7 ready
chr8 ready
chr9 ready
chr10 ready
chr11 ready
chr12 ready
chr13 ready
chr14 ready
chr15 ready
chr16 ready
chr17 ready
chr18 ready
chr19 ready
chr20 ready
chr21 ready
chr22 ready
chrM ready
chrX ready
chrY ready
Genome index ready in 'ref'


# Pré-traitement des données SRA

Les données de RNA-seq sont téléchargées et converties au format fastq, avant d'être alignées sur le génome.

In [ ]:
SRAIDS_MUTES = ["SRR628582", "SRR628583", "SRR628584"]
SRAIDS_NONMUTES = ["SRR628585", "SRR628586", "SRR628587", "SRR628588", "SRR628589"]
SRAIDS = SRAIDS_MUTES + SRAIDS_NONMUTES

### Fetch files and convert to FASTQ

In [18]:
@require("os", "urllib.request")
def sra2fastq(sraid):
    srafile = "sra/%s.sra" % sraid
    if not os.path.exists("%s_2.fastq.gz"%sraid):
        if not os.path.exists(srafile):
            urllib.request.urlretrieve("http://appliances.france-bioinformatique.fr/reprohackathon/%s.sra" % sraid, srafile)
        os.system("%s --gzip --split-files sra/%s.sra" % (EXE["fastq-dump"], sraid))
        os.unklink(srafile)
    return sraid

%time par.map_sync(sra2fastq, SRAIDS)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.5 µs


[None, None, None, None]

### Alignement

In [ ]:
staropts = " ".join(["--outSAMstrandField intronMotif",
     "--outFilterMismatchNmax 4",
     "--outFilterMultimapNmax 10",
     "--genomeDir %s" % gendir,
     "--readFilesIn {0}_1.fastq.gz {0}_2.fastq.gz",
    "--readFilesCommand zcat",
     "--runThreadN 10"
     "--outSAMunmapped None",
     "--outSAMtype BAM SortedByCoordinate",
     "--outStd BAM_SortedByCoordinate",
     "--genomeLoad NoSharedMemory",
     "--limitBAMsortRAM 3000000000"])

def align_sra(sraid):
    os.system("%s %s > %s.bam" % (EXE["STAR"], staropts.format(sraid), sraid))

for sraid in SRAIDS:
    align_sra(sraid)
    print("%s done" % sraid)
